In [ ]:
import functions_framework
import pandas as pd
import requests
import sqlalchemy
from datetime import datetime
from pytz import timezone

@functions_framework.http
# Main function
def airports(request):
    connection_string = connection()
    clean_and_send_airports_data(connection_string)
    return 'Data successfully added'

def connection():
    schema = "gans"
    host = "34.79.246.249"
    user = "root"
    password = "Arsenal2024"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

# Function to retrieve cities information from the database
def get_cities_info(connection_string):
    return pd.read_sql('cities_info', con=connection_string)

# Function to get airports information for cities
def cities_airports(table_of_cities):
    latitudes = list(table_of_cities['city_latitude'])
    longitudes = list(table_of_cities['city_longitude'])
    city_ids = list(table_of_cities['city_id'])
    list_for_df = []

    for index, value in enumerate(latitudes):
        city_id = city_ids[index]
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/"\
              f"location/{value}/{longitudes[index]}/km/50/10"
        querystring = {"withFlightInfoOnly": "true"}
        headers = {
            "x-rapidapi-key": API_KEY ,
            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }

        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            response_json = response.json()

            response_normalized = pd.json_normalize(response_json['items'])
            response_normalized['city_id'] = city_id
            list_for_df.append(response_normalized)

        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
        except requests.exceptions.RequestException as req_err:
            print(f"Request error occurred: {req_err}")
        except Exception as e:
            print(f"An error occurred: {e}")

    if list_for_df:
        airports_df = pd.concat(list_for_df, ignore_index=True)
    else:
        airports_df = pd.DataFrame()

    return airports_df

# Function to clean and send airports data to SQL
def clean_and_send_airports_data(connection_string):
    table_of_cities = get_cities_info(connection_string)
    airports_df = cities_airports(table_of_cities)

    if not airports_df.empty:
        airports_df = airports_df.drop(columns=['name', 'municipalityName', 'timeZone', 'countryCode', 'location.lat', 'location.lon'])
        airports_df.rename(columns={'icao': 'airport_code', 'iata': 'city_code', 'shortName': 'airport_name'}, inplace=True)
        airports_df.to_sql('airports', if_exists='append', con=connection_string, index=False)

